In [1]:
# 模型的数据加载与预处理
import config
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from torch.autograd import Function
from tqdm import tqdm
from utility_uad_svm import load_data, make_sequences, create_dataloaders, SeqDataset

#for key in config.csv_path:
#    print(f"数据集{key}路径: {config.csv_path[key]}")

--- [Config] 正在使用的设备: cpu ---


In [2]:
import torch
import torch.nn as nn
from torch.autograd import Function
import numpy as np

# === 1. 定义梯度反转层 (GRL) ===
class GradientReverseFunction(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        # 在前向传播中，不改变输入，但保存 alpha 用于反向传播
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        # 在反向传播中，将梯度乘以 -alpha
        output = grad_output.neg() * ctx.alpha
        return output, None

def grad_reverse(x, alpha):
    return GradientReverseFunction.apply(x, alpha)

In [3]:
def train_loop(G_f, G_y, G_d, dl_src, dl_tgt, opt_G, opt_D, loss_cls_fn, loss_dom_fn, device, epoch, total_epochs):
    """
    参数主要变更:
    1. 移除了 grl 参数 (我们直接在函数里用 grad_reverse)
    2. 增加了 epoch 和 total_epochs (用于计算进度 p)
    """
    G_f.train()
    G_y.train()
    G_d.train()
    
    total_loss_cls = 0.0
    total_loss_dom = 0.0
    
    # 获取批次总数用于计算进度
    len_dataloader = min(len(dl_src), len(dl_tgt)) 
    iter_src = iter(dl_src)
    iter_tgt = iter(dl_tgt) # 同时也建议把 target 做成 iter，防止长度不一致报错

    # TQDM 进度条
    from tqdm import tqdm
    pbar = tqdm(range(len_dataloader), desc=f"Epoch {epoch+1}/{total_epochs}", leave=False)

    for batch_idx in pbar:
        # ----------------------------------------------------
        # --- 0. 计算动态 Alpha (核心修改) ---
        # ----------------------------------------------------
        # p: 训练进度，从 0 慢慢变到 1
        p = float(batch_idx + epoch * len_dataloader) / (total_epochs * len_dataloader)
        # alpha: 梯度反转强度，从 0 慢慢变到 1
        alpha = 2. / (1. + np.exp(-10 * p)) - 1
        
        # 更新进度条显示当前的 alpha
        pbar.set_postfix({'alpha': f'{alpha:.4f}'})

        # ----------------------------------------------------
        # --- 步骤 A & B: 加载数据 ---
        # ----------------------------------------------------
        try:
            data_src = next(iter_src)
            data_tgt = next(iter_tgt)
        except StopIteration:
            # 防止迭代器耗尽
            iter_src = iter(dl_src)
            iter_tgt = iter(dl_tgt)
            data_src = next(iter_src)
            data_tgt = next(iter_tgt)

        X_src_batch, y_src_batch = data_src[0], data_src[1]
        X_tgt_batch = data_tgt[0] # target 域通常没有 label 或不用 label

        # ----------------------------------------------------
        # --- 步骤 C: NaN/INF 安全检查 & 设备传输 ---
        # ----------------------------------------------------
        X_src = X_src_batch.to(device)
        y_src = y_src_batch.to(device)
        X_tgt = X_tgt_batch.to(device)

        if torch.isnan(X_src).any() or torch.isnan(X_tgt).any():
            continue # 跳过坏数据

        # ====================================================
        # 第一阶段：优化 G_f (特征提取) 和 G_y (分类)
        # ====================================================
        opt_G.zero_grad() # 清空 G_f and G_y 的梯度
        
        # 1. 特征提取
        feat_src = G_f(X_src)
        feat_tgt = G_f(X_tgt)

        # 2. 类别分类损失 (仅源域)
        logits_cls = G_y(feat_src)
        loss_cls = loss_cls_fn(logits_cls, y_src)

        # 3. 域判别损失 (用于对抗)
        # [关键点] 这里应用 动态 GRL
        feat_src_adv = grad_reverse(feat_src, alpha)
        feat_tgt_adv = grad_reverse(feat_tgt, alpha)
        
        # 拼接用于判别器
        feat_combined_adv = torch.cat((feat_src_adv, feat_tgt_adv), dim=0)
        
        # 准备域标签 (0: Source, 1: Target)
        domain_label_src = torch.zeros(feat_src.size(0), dtype=torch.long, device=device)
        domain_label_tgt = torch.ones(feat_tgt.size(0), dtype=torch.long, device=device)
        domain_label_combined = torch.cat((domain_label_src, domain_label_tgt), dim=0)

        # 通过判别器
        logits_dom_adv = G_d(feat_combined_adv)
        loss_dom_adv = loss_dom_fn(logits_dom_adv, domain_label_combined)

        # 总损失：分类损失 + 域对抗损失
        # 注意：因为用了 grad_reverse，backward 时 loss_dom_adv 的梯度会自动反转
        # 所以这里是用加号 (+)
        loss_total_G = loss_cls + loss_dom_adv
        loss_total_G.backward()
        opt_G.step()

        # ====================================================
        # 第二阶段：优化 G_d (域判别器)
        # ====================================================
        # 这一步是为了让 G_d 尽可能准，不涉及 GRL，也不更新 G_f
        opt_D.zero_grad()

        with torch.no_grad():
            feat_src_fixed = G_f(X_src)
            feat_tgt_fixed = G_f(X_tgt)
        
        feat_combined_fixed = torch.cat((feat_src_fixed, feat_tgt_fixed), dim=0)
        logits_dom_fixed = G_d(feat_combined_fixed)
        loss_dom_D = loss_dom_fn(logits_dom_fixed, domain_label_combined)
        
        loss_dom_D.backward()
        opt_D.step()

        # ----------------------------------------------------
        # --- 记录数据 ---
        # ----------------------------------------------------
        total_loss_cls += loss_cls.item()
        total_loss_dom += loss_dom_D.item()

    return total_loss_cls / len_dataloader, total_loss_dom / len_dataloader

In [4]:
# ==========================================
# 1. 基础设置
# ==========================================
# 读取配置中的 ID 和 路径
SRC_IDS = config.SRC_IDS 
TGT_ID = config.TGT_ID    
CSV_PATHS = config.CSV_PATHS 

In [5]:
import config
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from models import LSTMFeatureExtractor, LabelClassifier, DomainDiscriminator
from utility_uad_svm import load_data, make_sequences, create_dataloaders, train_lstm_dann_standardization, lstm_standardization_train_pre_svm

# 引入训练循环函数 (假设它定义在同一个文件或 utility 中，如果是在本文件定义的，请保持不动)
# from your_file import train_loop 

def main():
    """
    [Main 函数详解]
    这是整个程序的控制中心。
    """
    # 设置计算设备 (GPU/CPU)
    device = torch.device(config.DEVICE)
    print(f"--- [Main] 正在使用设备: {device} ---")
    
    # ==========================================
    # 2. 数据准备 (Data Pipeline)
    # ==========================================
    print("\n--- [Main - 步骤 1-3] 正在加载并准备数据... ---")
    
    # -------------------------------------------------------------
    # [修复核心] 1. 加载所有源域数据以构建 df_src_all
    # -------------------------------------------------------------
    print(" -> 正在加载所有源域数据 (用于计算标准化统计量)...")
    df_src_list = []
    # 遍历所有源域 ID
    for src_id in config.SRC_IDS:
        path = config.CSV_PATHS[src_id]
        if os.path.exists(path):
            df_tmp = pd.read_csv(path)
            df_src_list.append(df_tmp)
        else:
            print(f"警告: 找不到文件 {path}")
            
    if not df_src_list:
        print("错误: 没有加载到任何源域数据！")
        return

    # 合并成一个大 DataFrame，这就是缺失的 df_src_all
    df_src_all = pd.concat(df_src_list, ignore_index=True)
    print(f" -> 源域数据加载完成，总行数: {len(df_src_all)}")

    # -------------------------------------------------------------
    # 2. 训练标准化器 (Fit & Save Scaler)
    # -------------------------------------------------------------
    # 这里调用第一个辅助函数：计算均值方差并保存 Scaler
    # 注意：这里的 df_src_all 只是用来计算参数的
    train_lstm_dann_standardization(
        df_src_all, 
        config.FEATURES, 
        config.MODEL_SAVE_DIR
    )
    # 注意：执行完这一步，config.MODEL_SAVE_DIR 下会生成 global_scaler.pkl
    
    # -------------------------------------------------------------
    # 3. 制作源域序列 (Load -> Standardize -> Sequence)
    # -------------------------------------------------------------
    print(" -> 正在制作源域序列 (逐个Case标准化并切片)...")
    all_X_list = []
    all_y_list = []
    
    # 重新遍历文件，这次是为了制作序列，同时应用刚才保存的 Scaler
    for src_id in config.SRC_IDS:
        path = config.CSV_PATHS[src_id]
        df_case = pd.read_csv(path)
        
        # [关键] 加载刚才存好的 Scaler 并应用标准化 (Transform only)
        # 这样确保每个 Case 都是用全局标准处理的
        df_case = lstm_standardization_train_pre_svm(
            df_case,
            config.FEATURES,
            config.MODEL_SAVE_DIR
        )
        
        # 制作序列
        X_c, y_c = make_sequences(
            df=df_case, 
            features=config.FEATURES, 
            target=config.TARGET_COL,
            seq_len=config.SEQ_LEN, 
            step=config.STEP
        )
        
        if len(X_c) > 0:
            all_X_list.append(X_c)
            all_y_list.append(y_c)

    if not all_X_list:
        print("错误: 源域序列生成失败。")
        return

    X_src = np.concatenate(all_X_list, axis=0)
    y_src = np.concatenate(all_y_list, axis=0)
    print(f" -> 源域序列制作完成。X_src: {X_src.shape}")

    # -------------------------------------------------------------
    # 4. 处理目标域数据 (Load -> Standardize -> Sequence)
    # -------------------------------------------------------------
    print(" -> 正在处理目标域数据...")
    _, df_tgt = load_data(config.SRC_IDS, config.TGT_ID, config.CSV_PATHS)
    
    if df_tgt is None: 
        print("错误：目标域数据加载失败。")
        return

    # 清洗 Inf/NaN
    df_tgt = df_tgt.fillna(0).replace([np.inf, -np.inf], 0)
    
    # [关键] 对目标域应用同样的标准化
    df_tgt = lstm_standardization_train_pre_svm(
        df_tgt, 
        config.FEATURES, 
        config.MODEL_SAVE_DIR
    )
    
    # 制作序列 (Target=None)
    X_tgt = make_sequences(
        df=df_tgt, 
        features=config.FEATURES, 
        target=None, 
        seq_len=config.SEQ_LEN, 
        step=config.STEP
    )
    
    if X_src is None or X_tgt is None: 
        print("错误：序列生成失败 (X_src 或 X_tgt 为空)。")
        return
        
    # -------------------------------------------------------------
    # 5. 创建 DataLoader
    # -------------------------------------------------------------
    dl_src, dl_tgt = create_dataloaders(X_src, y_src, X_tgt, config.BATCH_SIZE)
    if dl_src is None or dl_tgt is None: return
    
    print("--- [Main - 步骤 1-3] 数据准备完毕。---")
    
    # ==========================================
    # 3. 搭建模型 (Model Setup)
    # ==========================================
    print("\n--- [Main - 步骤 4] 正在搭建模型框架... ---")
    
    # G_f: 特征提取器
    G_f = LSTMFeatureExtractor(
        input_size=config.LSTM_INPUT_SIZE,   
        hidden_size=config.LSTM_HIDDEN_SIZE, 
        num_layers=config.LSTM_NUM_LAYERS,   
        dropout=config.LSTM_DROPOUT
    ).to(device)

    # G_y: 标签分类器
    G_y = LabelClassifier(
        input_size=config.LSTM_HIDDEN_SIZE, 
        num_classes=config.NUM_CLASSES,      
        hidden_dim=config.CLASSIFIER_HIDDEN_DIM
    ).to(device)

    # G_d: 域判别器
    G_d = DomainDiscriminator(
        input_size=config.LSTM_HIDDEN_SIZE, 
        hidden_dim=config.CLASSIFIER_HIDDEN_DIM
    ).to(device)
    
    print(" - 模型已创建。")

    # ==========================================
    # 4. 损失函数与优化器
    # ==========================================
    class_weights = torch.tensor([4.0, 1.0]).to(device) # 根据你的情况调整
    loss_cls_fn = nn.CrossEntropyLoss(weight=class_weights)
    loss_dom_fn = nn.CrossEntropyLoss()

    opt_G = optim.Adam(list(G_f.parameters()) + list(G_y.parameters()), lr=config.LEARNING_RATE_G)
    opt_D = optim.Adam(G_d.parameters(), lr=config.LEARNING_RATE_D)
    
    # ==========================================
    # 5. 训练循环 (Training Loop)
    # ==========================================
    print("\n--- [Main - 步骤 6] !!! 开始训练 !!! ---") 
    
    final_loss_cls = 0.0
    final_loss_dom = 0.0
    
    for epoch in range(config.NUM_EPOCHS):
        # 注意：这里需要你确保 train_loop 函数在作用域内
        # 如果 train_loop 在该文件外部定义，需要正确 import
        avg_loss_cls, avg_loss_dom = train_loop(
            G_f, G_y, G_d,          
            dl_src, dl_tgt,         
            opt_G, opt_D,           
            loss_cls_fn, loss_dom_fn, 
            device,                 
            epoch,                  
            config.NUM_EPOCHS       
        )
        
        print(f"Epoch [{epoch+1}/{config.NUM_EPOCHS}] 完成. Cls Loss: {avg_loss_cls:.4f} | Dom Loss: {avg_loss_dom:.4f}")
        final_loss_cls = avg_loss_cls
        final_loss_dom = avg_loss_dom
        
    print("\n--- [Main - 步骤 6] !!! 训练完成 !!! ---")
    
    # ==========================================
    # 6. 保存模型
    # ==========================================
    print("\n--- [Main - 步骤 7] 正在保存最终模型... ---") 
    os.makedirs(config.MODEL_SAVE_DIR, exist_ok=True) 
    
    g_f_path = os.path.join(config.MODEL_SAVE_DIR, "G_f_final.pth")
    g_y_path = os.path.join(config.MODEL_SAVE_DIR, "G_y_final.pth")
    
    torch.save(G_f.state_dict(), g_f_path)
    torch.save(G_y.state_dict(), g_y_path)
    
    print(f" - 特征提取器 (G_f) 已保存到: {g_f_path}")
    print(f" - 标签分类器 (G_y) 已保存到: {g_y_path}")

if __name__ == "__main__":
    main()

--- [Main] 正在使用设备: cpu ---

--- [Main - 步骤 1-3] 正在加载并准备数据... ---
 -> 正在加载所有源域数据 (用于计算标准化统计量)...
 -> 源域数据加载完成，总行数: 710063

[Standardization] 正在计算源域统计量并标准化 (Fit & Transform)...
 -> 特征均值 (前5个): [ 85.70710577  28.48481672  -0.32090063 -17.25298188   0.3716304 ]
 -> 特征方差 (前5个): [4.28103153e+03 4.20093196e+02 1.05978683e+03 2.84691761e+06
 2.33521247e-01]
 -> Scaler 已保存至: c:\Users\yangj\Desktop\GNSS-main\GNSS-main\Transfer Learning\SVM\checkpoints\global_scaler.pkl
 -> 正在制作源域序列 (逐个Case标准化并切片)...
  - X 形状: (91655, 5, 5), y 形状: (91655,)
  - X 形状: (91655, 5, 5), y 形状: (91655,)
  - X 形状: (79792, 5, 5), y 形状: (79792,)
  - X 形状: (79830, 5, 5), y 形状: (79830,)
  - X 形状: (91654, 5, 5), y 形状: (91654,)
  - X 形状: (91655, 5, 5), y 形状: (91655,)
  - X 形状: (91655, 5, 5), y 形状: (91655,)
  - X 形状: (91655, 5, 5), y 形状: (91655,)
 -> 源域序列制作完成。X_src: (709551, 5, 5)
 -> 正在处理目标域数据...
[load_data] 正在加载目标域数据: 0
[load_data] 目标域加载完成。形状: (12614, 14)
[load_data] 注：源域数据将在 'create_all_sequences' 中逐个加载以保证时序独立性。
  - X 形状: (12

Epoch [1/50] 完成. Cls Loss: 0.4176 | Dom Loss: 0.6569


Epoch [2/50] 完成. Cls Loss: 0.3632 | Dom Loss: 0.6399


Epoch [3/50] 完成. Cls Loss: 0.3354 | Dom Loss: 0.6984


Epoch [4/50] 完成. Cls Loss: 0.3006 | Dom Loss: 0.7080


...

Epoch [46/50] 完成. Cls Loss: 0.2310 | Dom Loss: 0.6911


Epoch [47/50] 完成. Cls Loss: 0.2361 | Dom Loss: 0.6859


Epoch [48/50] 完成. Cls Loss: 0.2331 | Dom Loss: 0.6890


Epoch [49/50] 完成. Cls Loss: 0.2259 | Dom Loss: 0.6910


Epoch [50/50] 完成. Cls Loss: 0.2301 | Dom Loss: 0.6838

--- [Main - 步骤 6] !!! 训练完成 !!! ---

--- [Main - 步骤 7] 正在保存最终模型... ---
 - 特征提取器 (G_f) 已保存到: c:\Users\yangj\Desktop\GNSS-main\GNSS-main\Transfer Learning\SVM\checkpoints\G_f_final.pth
 - 标签分类器 (G_y) 已保存到: c:\Users\yangj\Desktop\GNSS-main\GNSS-main\Transfer Learning\SVM\checkpoints\G_y_final.pth


In [6]:
# from models import LSTMFeatureExtractor, LabelClassifier, DomainDiscriminator, GRL_Layer
# import torch.optim as optim
# from utility_uad_svm  import load_data, make_sequences, create_dataloaders,create_all_sequences
# from utility_uad_svm import train_lstm_dann_standardization, lstm_standardization_train_pre_svm
# def main():
#     """
#     [Main 函数详解]
#     这是整个程序的控制中心。
#     """
#     # ... 在 main 函数里 ...

#     # 1. 对源域 (Fit + Save)
#     df_src_all = train_lstm_dann_standardization(
#         df_src_all, 
#         config.FEATURES, 
#         config.MODEL_SAVE_DIR
#     )

#     # 2. 对目标域 (Transform only - 虽然 DANN 训练时不一定要用到目标域的特征值本身，但保持一致比较好)
#     df_tgt = lstm_standardization_train_pre_svm(
#         df_tgt, 
#         config.FEATURES, 
#         config.MODEL_SAVE_DIR
#     )
    
#     # 设置计算设备 (GPU/CPU)
#     device = torch.device(config.DEVICE)
#     print(f"--- [Main] 正在使用设备: {device} ---")
    
#     # ==========================================
#     # 2. 数据准备 (Data Pipeline)
#     # ==========================================
#     print("\n--- [Main - 步骤 1-3] 正在加载并准备数据... ---")
    
#     # 2a. 加载原始 CSV 数据
#     df_src, df_tgt = load_data(SRC_IDS, TGT_ID, CSV_PATHS)
#     # if df_src is None or df_tgt is None: return
#     if df_tgt is None: 
#         print("错误：目标域数据加载失败。")
#         return




#     # 2b. 简单的 NaN/Inf 清洗 (针对目标域)
#     if df_tgt.isnull().values.any():
#         print(f"【警告】目标域有 NaN，正在填充 0...")
#         df_tgt = df_tgt.fillna(0)
#     if (df_tgt == np.inf).any().any() or (df_tgt == -np.inf).any().any():
#         print("【警告】目标域有 INF，正在替换为 0...")
#         df_tgt = df_tgt.replace([np.inf, -np.inf], 0)

#     # 2c. 制作时间序列 (Sequences)
#     # 这里调用 create_all_sequences，它内部会调用 make_sequences

#     X_src, y_src, X_tgt = create_all_sequences(df_src, df_tgt, config)
#     # [新增] 安全检查：确保序列真的生成了
#     if X_src is None or X_tgt is None: 
#         print("错误：序列生成失败 (X_src 或 X_tgt 为空)。")
#         return
#     # if X_src is None or X_tgt is None: return
        
#     # 2d. 创建 DataLoader (投喂器)
#     # 这一步把 numpy 数组变成了 PyTorch 可以批量读取的对象
#     dl_src, dl_tgt = create_dataloaders(X_src, y_src, X_tgt, config.BATCH_SIZE)
#     if dl_src is None or dl_tgt is None: return
    
#     print("--- [Main - 步骤 1-3] 数据准备完毕。---")
    
#     # ==========================================
#     # 3. 搭建模型 (Model Setup)
#     # ==========================================
#     print("\n--- [Main - 步骤 4] 正在搭建模型框架... ---")
    
#     # (4a) G_f: 特征提取器
#     # 假设您修改后的 __init__ 默认 final_feature_dim=128，或者跟 hidden_size 一样
#     G_f = LSTMFeatureExtractor(
#         input_size=config.LSTM_INPUT_SIZE,   # 例如 5 (根据特征数量)
#         hidden_size=config.LSTM_HIDDEN_SIZE, # 例如 128 (内部 LSTM 单元数)
#         num_layers=config.LSTM_NUM_LAYERS,   # 例如 2
#         dropout=config.LSTM_DROPOUT
#         # 如果您代码里加了 final_feature_dim 参数，这里最好显式传一下，例如：
#         # final_feature_dim=config.LSTM_HIDDEN_SIZE 
#     ).to(device)

#     # (4b) G_y: 标签分类器
#     # 接收 G_f 的输出。只要 G_f 输出是 128，这里 input_size=128 就没问题
#     G_y = LabelClassifier(
#         input_size=config.LSTM_HIDDEN_SIZE, 
#         num_classes=config.NUM_CLASSES,      # 例如 2 (好/坏)
#         hidden_dim=config.CLASSIFIER_HIDDEN_DIM
#     ).to(device)

#     # (4c) G_d: 域判别器
#     # 接收 G_f 的输出。道理同上。
#     G_d = DomainDiscriminator(
#         input_size=config.LSTM_HIDDEN_SIZE, 
#         hidden_dim=config.CLASSIFIER_HIDDEN_DIM
#     ).to(device)

#     # (4d) 【重要修改】删除 GRL 层的实例化
#     # grl = GRL_Layer(...)  <--- 删除这行！
#     # 原因：我们在 train_loop 里使用动态计算的 alpha，不需要这个固定的层了。
    
#     print(" - G_f, G_y, G_d 模型已在设备上创建。")

#     # ==========================================
#     # 4. 损失函数与优化器
#     # ==========================================
#     print("\n--- [Main - 步骤 5] 正在搭建损失函数和优化器... ---")

#     # (5a) 损失函数
#     class_weights = torch.tensor([4.0, 1.0]).to(device) 
#     loss_cls_fn = nn.CrossEntropyLoss(weight=class_weights)
#     loss_dom_fn = nn.CrossEntropyLoss() # 域分类通常是平衡的，不需要加权

#     # (5b) 优化器
#     # 优化 G_f 和 G_y
#     opt_G = optim.Adam(
#         list(G_f.parameters()) + list(G_y.parameters()),
#         lr=config.LEARNING_RATE_G
#     )
#     # 独立优化 G_d
#     opt_D = optim.Adam(
#         G_d.parameters(),
#         lr=config.LEARNING_RATE_D
#     )
    
#     print(" - 损失函数和优化器已定义。")
#     print("--- [Main - 步骤 4 & 5] 模型搭建完毕。---")
    
#     # ==========================================
#     # 5. 训练循环 (Training Loop)
#     # ==========================================
#     print("\n--- [Main - 步骤 6] !!! 开始训练 !!! ---") 
    
#     final_loss_cls = 0.0
#     final_loss_dom = 0.0
    
#     for epoch in range(config.NUM_EPOCHS):
        
#         # 【关键修改】调用 train_loop
#         # 1. 删除了 grl 参数
#         # 2. 传入了 epoch 和 config.NUM_EPOCHS 用于计算动态 alpha
#         avg_loss_cls, avg_loss_dom = train_loop(
#             G_f, G_y, G_d,          # 模型
#             dl_src, dl_tgt,         # 数据
#             opt_G, opt_D,           # 优化器
#             loss_cls_fn, loss_dom_fn, # 损失函数
#             device,                 # 设备
#             epoch,                  # 当前轮数 (新!)
#             config.NUM_EPOCHS       # 总轮数 (新!)
#         )
        
#         # 打印进度 (train_loop 里已经有进度条，这里打印 Epoch 总结)
#         print(f"Epoch [{epoch+1}/{config.NUM_EPOCHS}] 完成. Cls Loss: {avg_loss_cls:.4f} | Dom Loss: {avg_loss_dom:.4f}")
        
#         final_loss_cls = avg_loss_cls
#         final_loss_dom = avg_loss_dom
        
#         # (可选) 在这里可以加一个 save_checkpoint 的逻辑
        
#     print("\n--- [Main - 步骤 6] !!! 训练完成 !!! ---")
#     print(f" - 最终分类损失 (Loss_cls): {final_loss_cls:.4f}")
#     print(f" - 最终域损失 (Loss_dom): {final_loss_dom:.4f}")
    
#     # ==========================================
#     # 6. 保存模型
#     # ==========================================
#     print("\n--- [Main - 步骤 7] 正在保存最终模型... ---") 
#     os.makedirs(config.MODEL_SAVE_DIR, exist_ok=True) 
    
#     g_f_path = os.path.join(config.MODEL_SAVE_DIR, "G_f_final.pth")
#     g_y_path = os.path.join(config.MODEL_SAVE_DIR, "G_y_final.pth")
    
#     torch.save(G_f.state_dict(), g_f_path)
#     torch.save(G_y.state_dict(), g_y_path)
    
#     print(f" - 特征提取器 (G_f) 已保存到: {g_f_path}")
#     print(f" - 标签分类器 (G_y) 已保存到: {g_y_path}")

# # 别忘了运行 main
# if __name__ == "__main__":
#     main()
  
